## 소개
네이버증권에서 종목리스트를 추출하고  
내일주가로 UP/DOWN을 예측합니다.

## 종목명 1
네이버증권 테마

In [ ]:
import naver_theme_scraping
items = naver_theme_scraping.get_items()
links = naver_theme_scraping.get_links(items)
firms = naver_theme_scraping.get_firms(links)

In [ ]:
firm_names = firms
print(firm_names[:5])

## 종목명 2 
네이버증권 등락률상위, 등락률하위, 거래량상위

In [1]:
import top_momentum_stocks

In [2]:
firm_names = top_momentum_stocks.top_momentum_stocks()

C:\Users\user\Desktop\nextupdown\top_momentum_stocks.py:12: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 12 of the file C:\Users\user\Desktop\nextupdown\top_momentum_stocks.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(res.text, 'html')
C:\Users\user\Desktop\nextupdown\top_momentum_stocks.py:22: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that ca

In [3]:
print(firm_names[:5])

['천보', 'TRUE 블룸버그 인버스 2X 천연가스선물 ETN', '메리츠 천연가스 선물 ETN(H)', '팜스토리', '대신 S&P 인버스 2X 천연가스 선물 ETN']


## 종목코드
종목명을 종목코드로 변환합니다.

In [4]:
import firm_code
firm_codes = firm_code.firm_codes(firm_names)
firm_codes = firm_code.filtering_firm_codes(firm_codes)

In [5]:
print(firm_codes[:5])

['278280', '027710', '200710', '072130', '024850']


## 내일주가

In [7]:
import FinanceDataReader as fdr
import nextprice_preprocessing
import pandas as pd

In [8]:
def concat_firms():
    frame = pd.DataFrame()
    for firm_code in firm_codes:
        df = fdr.DataReader(symbol=firm_code)
        df = nextprice_preprocessing.moving_average(df, change=True)
        df = nextprice_preprocessing.changing_price(df)
        df = nextprice_preprocessing.create_y_label(df)
        df = nextprice_preprocessing.slice_year(df,'2015')
        frame = pd.concat([frame, df])

    return frame

In [9]:
df = concat_firms()

In [10]:
df.tail(3)

,Open,High,Low,Close,Volume,Change,MA5,MA20,MA60,MA120,y
Date,,,,,,,,,,,
2023-02-20,-0.069343,0.074818,-0.069343,5480,728205,0.062016,0.050365,0.066195,0.088869,-0.009542,1
2023-02-21,-0.005357,0.048214,-0.030357,5600,347429,0.021898,0.051786,0.079062,0.106905,0.016749,1
2023-02-22,-0.099836,0.063830,-0.101473,6110,1878521,0.091071,0.098854,0.145908,0.178642,0.101834,1


In [11]:
len(df)

119133

In [12]:
result = df.isna().sum()
result

Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
MA5       0
MA20      0
MA60      0
MA120     0
y         0
dtype: int64

In [13]:
df.dropna(inplace=True)

In [14]:
result = df.isna().sum()
result

Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
MA5       0
MA20      0
MA60      0
MA120     0
y         0
dtype: int64

In [15]:
X, y = nextprice_preprocessing.split_x_y(df)

In [16]:
X.tail(3)

,MA5,MA20,MA60,MA120,Volume,Change,Open,High,Low
Date,,,,,,,,,
2023-02-17,0.006977,0.014486,0.033689,-0.076898,54035,-0.003861,0.000000,0.011628,-0.017442
2023-02-20,0.050365,0.066195,0.088869,-0.009542,728205,0.062016,-0.069343,0.074818,-0.069343
2023-02-21,0.051786,0.079062,0.106905,0.016749,347429,0.021898,-0.005357,0.048214,-0.030357


In [17]:
y.tail(3)

Date
2023-02-20    1
2023-02-21    1
2023-02-22    1
Name: y, dtype: int32

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Scaler

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## PCA

In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

## Model

In [21]:
from keras.optimizers import Adagrad
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense

In [22]:
model = Sequential()
model.add(Dense(12, input_dim=3, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adagrad(learning_rate=0.2)

In [23]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                48        
                                                                 
 dropout (Dropout)           (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 12)                156       
                                                                 
 dropout_1 (Dropout)         (None, 12)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 217
Trainable params: 217
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train, y_train, epochs=30, batch_size=20)

Epoch 1/30
4468/4468 [==============================] - 8s 2ms/step - loss: 0.6789 - accuracy: 0.5566
Epoch 2/30
4468/4468 [==============================] - 7s 2ms/step - loss: 0.6778 - accuracy: 0.5579
Epoch 3/30
4468/4468 [==============================] - 6s 1ms/step - loss: 0.6775 - accuracy: 0.5584
Epoch 4/30
4468/4468 [==============================] - 6s 1ms/step - loss: 0.6772 - accuracy: 0.5588
Epoch 5/30
4468/4468 [==============================] - 7s 1ms/step - loss: 0.6771 - accuracy: 0.5584
Epoch 6/30
4468/4468 [==============================] - 7s 2ms/step - loss: 0.6774 - accuracy: 0.5586
Epoch 7/30
4468/4468 [==============================] - 9s 2ms/step - loss: 0.6772 - accuracy: 0.5584
Epoch 8/30
4468/4468 [==============================] - 6s 1ms/step - loss: 0.6770 - accuracy: 0.5587
Epoch 9/30
4468/4468 [==============================] - 7s 2ms/step - loss: 0.6768 - accuracy: 0.5596
Epoch 10/30
4468/4468 [==============================] - 7s 1ms/step - loss: 0.676

## Evaluation

In [25]:
_, accuracy = model.evaluate(X_test, y_test)
print(accuracy)

931/931 [==============================] - 1s 1ms/step - loss: 0.6769 - accuracy: 0.5624
0.56243497133255


In [26]:
y_pred = model.predict(X_test)

931/931 [==============================] - 1s 1ms/step


In [27]:
from sklearn import metrics
import numpy as np

In [28]:
confusion_matrix = metrics.confusion_matrix(y_test, np.rint(y_pred))
confusion_matrix

array([[16138,   477],
       [12555,   613]], dtype=int64)

## Save

In [29]:
import joblib
joblib.dump(scaler, './database/nextprice_scaler.pkl')

['./database/nextprice_scaler.pkl']

In [30]:
import pickle
pickle.dump(pca, open('./database/nextprice_pca.pkl','wb'))

In [31]:
from tensorflow import keras
model.save('./database/nextprice_model')

INFO:tensorflow:Assets written to: ./database/nextprice_model\assets


## Load

In [32]:
import joblib
scaler = joblib.load('./database/nextprice_scaler.pkl')

In [33]:
import pickle
pca = pickle.load(open('./database/nextprice_pca.pkl','rb'))

In [34]:
from tensorflow import keras
model = keras.models.load_model('./database/nextprice_model')

## Meta Cognition

In [35]:
print(accuracy)

0.56243497133255


In [36]:
firm_accuracy = []
for firm_code in firm_codes:
    df = fdr.DataReader(symbol=firm_code)
    df = nextprice_preprocessing.moving_average(df, change=True)
    df = nextprice_preprocessing.changing_price(df)
    df = nextprice_preprocessing.create_y_label(df)
    df = nextprice_preprocessing.slice_year(df,'2015')
    X, y = nextprice_preprocessing.split_x_y(df)
    X = scaler.transform(X)
    X = pca.transform(X)
    loss, acc = model.evaluate(X, y, batch_size=20)
    firm_accuracy.append(acc)

28/28 [==============================] - 0s 2ms/step - loss: 0.6848 - accuracy: 0.5727


In [37]:
df = pd.DataFrame({'firm_code':firm_codes, 'accuracy':firm_accuracy})

In [38]:
df.head()

,firm_code,accuracy
0,278280,0.513541
1,027710,0.551173
2,200710,0.552172
3,072130,0.541188
4,024850,0.558163


In [39]:
df = df[df['accuracy'] > accuracy]

In [40]:
df.head()

,firm_code,accuracy
5,208370,0.569646
9,042040,0.570145
17,016380,0.577134
18,208340,0.580553
19,089530,0.578632


In [41]:
firm_codes = df['firm_code'].tolist()

In [42]:
firm_codes[:5]

['208370', '042040', '016380', '208340', '089530']

## Prediction

In [43]:
def prediction():
    predictions = []
    for firm_code in firm_codes:
        df = fdr.DataReader(symbol=firm_code)
        df = nextprice_preprocessing.moving_average(df, change=True)
        df = nextprice_preprocessing.changing_price(df)
        df = df[['MA5', 'MA20', 'MA60', 'MA120', 'Volume', 'Change', 'Open', 'High', 'Low']]
        df = df[-1:]
        df = scaler.transform(df)
        df = pca.transform(df)
        prediction = model.predict(df)
        predictions.append([firm_code, prediction])

    return predictions

In [44]:
nextupdown = prediction()

1/1 [==============================] - 0s 24ms/step


In [45]:
nextupdown[0]

['208370', array([[0.44739926]], dtype=float32)]

In [46]:
df_nextupdown = pd.DataFrame(nextupdown, columns=['code','prediction'])

In [47]:
df_nextupdown.head(3)

,code,prediction
0,208370,[[0.44739926]]
1,042040,[[0.5061778]]
2,016380,[[0.52617204]]


In [48]:
import numpy as np
df_nextupdown['prediction'] = df_nextupdown['prediction'].apply(lambda x : np.rint(x))

In [49]:
df_nextupdown.head()

,code,prediction
0,208370,[[0.0]]
1,042040,[[1.0]]
2,016380,[[1.0]]
3,208340,[[0.0]]
4,089530,[[1.0]]


In [50]:
df_nextupdown[df_nextupdown['prediction'] == 1]

,code,prediction
1,042040,[[1.0]]
2,016380,[[1.0]]
4,089530,[[1.0]]
5,279600,[[1.0]]
6,328380,[[1.0]]
10,069920,[[1.0]]
13,344860,[[1.0]]
16,363260,[[1.0]]
18,014990,[[1.0]]
19,046140,[[1.0]]


In [51]:
df_code = pd.read_csv('./database/stock_code.csv', dtype={'종목코드':str})
df_code.head(1)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,000210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시


In [52]:
df_nextupdown['code'].isin(df_code['종목코드'])

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21    False
22     True
23     True
24     True
25     True
26     True
27    False
28     True
29     True
30     True
Name: code, dtype: bool

In [53]:
merged_df = df_nextupdown.merge(df_code, left_on='code', right_on='종목코드', how='left')

In [54]:
merged_df = merged_df[['code','회사명','prediction']]

In [55]:
merged_df[merged_df['prediction'] == 1]

,code,회사명,prediction
1,042040,케이피엠테크,[[1.0]]
2,016380,KG스틸,[[1.0]]
4,089530,에이티세미콘,[[1.0]]
5,279600,미디어젠,[[1.0]]
6,328380,미래에셋대우스팩3호,[[1.0]]
10,069920,아이에스이커머스,[[1.0]]
13,344860,이노진,[[1.0]]
16,363260,하나금융17호스팩,[[1.0]]
18,014990,인디에프,[[1.0]]
19,046140,SBS콘텐츠허브,[[1.0]]


In [56]:
merged_df['prediction'] = merged_df['prediction'].replace({1: 'UP', 0: 'DOWN'})

In [57]:
merged_df[merged_df['prediction'] == 'UP']

,code,회사명,prediction
1,042040,케이피엠테크,UP
2,016380,KG스틸,UP
4,089530,에이티세미콘,UP
5,279600,미디어젠,UP
6,328380,미래에셋대우스팩3호,UP
10,069920,아이에스이커머스,UP
13,344860,이노진,UP
16,363260,하나금융17호스팩,UP
18,014990,인디에프,UP
19,046140,SBS콘텐츠허브,UP
